In [385]:

pip install webdriver-manager


Note: you may need to restart the kernel to use updated packages.


In [386]:
pip install selenium

In [387]:
import numpy as np
import requests as requests
import webdriver_manager
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.chrome import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager import driver
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver





In [388]:
def scrape_movie_years_links(wiki_url):
    years = []
    link_to_film_of_year = []
    for year_link in bs.findAll("li"):
        if "List of" in year_link.text and "s" not in year_link.text[-4:]:
            if 1970 <= int(year_link.text[-4:]) <= 2020:
                link_to_film_of_year.append(year_link.find('a')['href'])
                years.append(year_link.text[-4:])
    return pd.DataFrame({"year": years, "link_to_year": link_to_film_of_year})


In [389]:
def load_movies_per_years(year_url_address, driver):
    movie_name = []
    years = []
    length = []
    idbm_year = []
    genres = []
    rating = []
    stars = []
    writers = []
    directors = []
    for item in range(year_url_address.shape[0]):
        next = year_url_address.iloc[item][1]
        response = requests.get("https://en.wikipedia.org/" + next)
        soup = BeautifulSoup(response.text, "html.parser")
        try:
            tables = soup.find_all("table", class_="wikitable")
            for table in tables:
                trs = table.find_all("tr")
                for tr in trs[1:]:
                    movie_data = tr.find_all("i")
                    add_data_to_table_selenium(movie_data[0].text.strip(), driver, directors, writers, stars, rating,
                                               genres, idbm_year, length)
                    movie_name.append(movie_data[0].text.strip())
                    years.append(year_url_address.iloc[item][0])

        except:
            print("exeption")
            continue
    driver.close()
    return pd.DataFrame(
        {"name": movie_name, "year": years, "idbm_year": idbm_year, "director": directors, "writers": writers,
         "star": stars, "rating": rating, "genre": genres})


In [390]:
def add_data_to_table_selenium(movie_name, s, directors, writers, stars, rating, genres, idbm_year):
    list_of_directors = []
    list_of_writers = []
    list_of_genres = []
    list_of_stars = []
    try:
        driver.get('https://www.imdb.com/')
        inputElement = driver.find_element_by_id("suggestion-search")
        inputElement.send_keys(movie_name)
        click_btn = driver.find_element_by_id("suggestion-search-button")
        click_btn.click()
        results = driver.find_element_by_xpath("//table[@class='findList']/tbody/tr/td[@class='primary_photo']/a")
        results.click()  # clicks the first one
        c = driver.page_source
        bs = BeautifulSoup(c, "html.parser")
    except:
        print("connection lost")
        genres.append("")
        rating.append("")
        directors.append("")
        writers.append("")
        stars.append("")
        idbm_year.append("")
        length.append("")
        return

    try:
        genre = bs.find("div", class_="ipc-chip-list").find_all('span', class_="ipc-chip__text")
        for g in genre:
            list_of_genres.append(g.text.strip())
        string_genres = ','.join(list_of_genres)
        genres.append(string_genres)
    except:
        genres.append("")
    try:
        year = bs.find("div", class_="TitleBlock__TitleMetaDataContainer-sc-1nlhx7j-2").find_all('li',
                                                                                                 class_="ipc-inline-list__item")
        y = year[0].find('span', class_='TitleBlockMetaData__ListItemText-sc-12ein40-2')
        idbm_year.append(y.text)
    except:
        idbm_year.append("")

    try:
        rate = bs.find("span", class_="AggregateRatingButton__RatingScore-sc-1ll29m0-1")
        rating.append(rate.text.strip())
    except:
        rating.append("")
    try:
        hour = bs.find("div", class_="TitleBlock__TitleMetaDataContainer-sc-1nlhx7j-2 hWHMKr").find('ul',
                                                                                                 class_="ipc-inline-list").find_all('li',class_="ipc-inline-list__item")
        h = hour[2]
        print(h.text)
        length.append(h.text)
    except:
        length.append("")

    try:
        info_director_writer_stars = bs.find('div',
                                             class_="PrincipalCredits__PrincipalCreditsPanelWideScreen-sc-hdn81t-0").find_all(
            'ul', class_='ipc-inline-list')
        director = info_director_writer_stars[0].find_all('a',
                                                          class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')
        writer = info_director_writer_stars[1].find_all('a',
                                                        class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')
        star = info_director_writer_stars[2].find_all('a',
                                                      class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')
        for d in range(len(director)):
            list_of_directors.append(director[d].text)
        string_directors = ','.join(list_of_directors)
        directors.append(string_directors)
        for w in range(len(writer)):
            list_of_writers.append(writer[w].text)
        string_writers = ','.join(list_of_writers)
        writers.append(string_writers)
        for s in range(len(star)):
            list_of_stars.append(star[s].text)
        string_stars = ','.join(list_of_stars)
        stars.append(string_stars)

    except:
        directors.append("")
        writers.append("")
        stars.append("")


In [391]:
def load_csv(file_name):
    df = pd.read_csv(file_name)
    return df

In [7]:

list_of_american_films_wiki = "https://en.wikipedia.org/wiki/Lists_of_American_films"
r = requests.get(list_of_american_films_wiki)
bs = BeautifulSoup(r.content, "html.parser")
links_df = scrape_movie_years_links(bs.contents)
driver = webdriver.Chrome(ChromeDriverManager().install())
df = load_movies_per_years(links_df, driver)
df.to_csv('movie_wiki_done.csv')

NameError: name 'BeautifulSoup' is not defined

In [412]:
df =load_csv('movie_wiki_done.csv')
df.head(10)

,Unnamed: 0,name,year,idbm_year,length,director,writers,star,rating,genre
0,0,Adam at Six A.M.,1970,1970,1h 40m,Robert Scheerer,"Stephen Karpf,Elinor Karpf","Michael Douglas,Lee Purcell,Joe Don Baker",5.7,Drama
1,1,Adam's Woman,1970,1970,1h 56m,Philip Leacock,"Richard Fielder,Lowell Barrington","Beau Bridges,John Mills,Jane Merrow",6.3,Drama
2,2,The Adventurers,1970,1970,2h 51m,Lewis Gilbert,"Michael Hastings,Lewis Gilbert,Harold Robbins","Charles Aznavour,Alan Badel,Candice Bergen",5.4,"Action,Adventure,Drama"
3,3,Airport,1970,1970,2h 17m,"George Seaton,Henry Hathaway","Arthur Hailey,George Seaton","Burt Lancaster,Dean Martin,George Kennedy",6.6,"Action,Drama,Thriller"
4,4,Alex in Wonderland,1970,1970,1h 50m,Paul Mazursky,"Paul Mazursky,Larry Tucker","Donald Sutherland,Ellen Burstyn,Meg Mazursky",5.4,"Comedy,Drama"
5,5,The Angel Levine,1970,1970,1h 44m,Ján Kadár,"Bill Gunn,Ronald Ribman,Bernard Malamud","Zero Mostel,Harry Belafonte,Ida Kaminska",6.0,Drama
6,6,Angel Unchained,1970,1970,1h 26m,Lee Madden,"Jeffrey Alan Fiskin,Lee Madden","Don Stroud,Luke Askew,Larry Bishop",5.1,"Action,Thriller,Drama"
7,7,The Aristocats,1970,1970,1h 18m,Wolfgang Reitherman,"Larry Clemmons,Vance Gerry,Ken Anderson","Phil Harris,Eva Gabor,Sterling Holloway",7.1,"Animation,Adventure,Comedy"
8,8,The Baby Maker,1970,1970,1h 49m,James Bridges,James Bridges,"Barbara Hershey,Collin Wilcox Paxton,Sam Groom",6.0,Drama
9,9,The Ballad of Cable Hogue,1970,1970,2h 1m,Sam Peckinpah,"John Crawford,Edmund Penney,Gordon T. Dawson","Jason Robards,Stella Stevens,David Warner",7.3,"Comedy,Drama,Romance"


In [413]:
def list_of_women(bs, womens_list):
    womens = []
    link_to_film_of_year = []
    divs = bs.findAll("div", class_="div-col")
    for div in divs:
        li = div.findAll('li')
        for l in li:
            women = l.text.split("(")
            womens_list.append(women[0].strip()) 
    

In [414]:
def list_of_women_from_imdb_by_film(bs, womens_list):
    next = "?page=1&keywords=female-director&mode=detail&ref_=kw_nxt&sort=moviemeter,asc"
    for i in range(4):
        r = requests.get("https://www.imdb.com/search/keyword/" + next)
        bs = BeautifulSoup(r.content, "html.parser")
        list_of_divs = bs.find("div", class_="lister-list").findAll("div", class_='lister-item mode-detail')
        for div in list_of_divs:
            p_links = div.find("div", class_="lister-item-content").findAll("p", class_="text-muted")
            a_links = p_links[1].findAll("a")
            try:
                womens_list.append(a_links[0].text)
            except:
                continue
        try:
            next = bs.find('a', class_='lister-page-next next-page')['href']
        except:
            continue

  

    

In [415]:
def list_of_women_writers(bs, womens_list):
    womens = []
    link_to_film_of_year = []
    divs = bs.findAll("div", class_="div-col")
    for div in divs:
        li = div.findAll('li')
        for l in li:
            women = l.text.split("(")
            womens_list.append(women[0].strip()) 

In [416]:
def scrape_movie_producer_links(wiki_url):
    link_to_film_writer_categories = []
    categories = bs.find("div", class_="mw-category").findAll("div", class_="mw-category-group")
    for category in categories:
        links_to__writer_categories = category.findAll("li")
        for link in links_to__writer_categories:
            link_to_film_writer_categories.append(link.find("a").text)
    return link_to_film_writer_categories



In [417]:
def get_producer_women(links_producer_women, womens_list):
    for link in links_producer_women:
        url = 'https://en.wikipedia.org/wiki/Category:'+ link
        r = requests.get(url)
        bs = BeautifulSoup(r.content, "html.parser")
        categories = bs.find("div", class_="mw-category-generated").find("div", class_="mw-content-ltr").findAll("ul")
        for category in categories:
            names = category.findAll("li")
            for name in names:
                a_ = name.findAll("a")
                for a in a_:
                    name = a.text.split("(")
                    womens_list.append(name[0].strip())

In [418]:
def scrape_movie_actors_links(wiki_url):
    link_to_film_actors_categories = []
    categories = bs.find("div", class_="hlist hlist-separated").findAll("li")
    for category in categories:
        link_to_film_actors_categories.append(category.text)
    return link_to_film_actors_categories



In [419]:
def get_actors_women(links_actors_women, womens_list):
    for link in links_actors_women:
        url = 'https://en.wikipedia.org/wiki/List_of_female_movie_actors_by_name:_' + link
        r = requests.get(url)
        bs = BeautifulSoup(r.content, "html.parser")
        uls = bs.findAll("ul")
        for ul in uls[1:-15]:
            lis = ul.findAll("li")
            for i in lis:
                name = i.text.split('(')
                womens_list.append(name[0].strip())

In [420]:
womens_list = []

list_of_womens_director = "https://en.wikipedia.org/wiki/List_of_female_film_and_television_directors"
r = requests.get(list_of_womens_director)
bs = BeautifulSoup(r.content, "html.parser")
list_of_women(bs,womens_list)

list_of_womens_writer1 = "https://en.wikipedia.org/wiki/List_of_women_writers_(A%E2%80%93L)"
r = requests.get(list_of_womens_writer1)
bs = BeautifulSoup(r.content, "html.parser")
list_of_women_writers(bs,womens_list)

list_of_womens_writer2 = "https://en.wikipedia.org/wiki/List_of_women_writers_(M%E2%80%93Z)"
r = requests.get(list_of_womens_writer2)
bs = BeautifulSoup(r.content, "html.parser")
list_of_women_writers(bs,womens_list)

list_of_womens_director1 = "https://www.imdb.com/search/keyword/?page=1&keywords=female-director&mode=detail&ref_=kw_nxt&sort=moviemeter,asc"
r = requests.get(list_of_womens_director1)
bs = BeautifulSoup(r.content, "html.parser")
list_of_women_from_imdb_by_film(bs, womens_list)

list_of_womens_producer1 = "https://en.wikipedia.org/wiki/Category:Women_film_producers"
r = requests.get(list_of_womens_producer1)
bs = BeautifulSoup(r.content, "html.parser")
links_producer_women = scrape_movie_producer_links(bs)
get_producer_women(links_producer_women, womens_list)

list_of_womens_actors = "https://en.wikipedia.org/wiki/List_of_female_movie_actors_by_name:_A"
r = requests.get(list_of_womens_actors)
bs = BeautifulSoup(r.content, "html.parser")
links_actors_women = scrape_movie_actors_links(bs)
get_actors_women(links_actors_women, womens_list)

print("done")



done


In [421]:
def divide_columns(df_before_split):
    star_col = df_before_split["star"].fillna("")
    director_col = df_before_split["director"].fillna("")
    writer_col = df_before_split["writers"].fillna("")
    genre_col = df_before_split["genre"].fillna("")
    
    stars_1 = []
    stars_2 = []
    stars_3 = []
    writers_1 = []
    writers_2 = []
    writers_3 = []
    directors_1 = []
    directors_2 = []
    directors_3 = []
    genres_1 = []
    genres_2 = []
    genres_3 = []

    for row in star_col:
        stars = row.split(",")
        if len(stars) < 3:
            for i in range((3 - len(stars))):
                stars.append("")
        stars_1.append(stars[0])
        stars_2.append(stars[1])
        stars_3.append(stars[2])

    for row in director_col:
        directors = row.split(",")
        if len(directors) < 3:
            for i in range((3 - len(directors))):
                directors.append("")
        directors_1.append(directors[0])
        directors_2.append(directors[1])
        directors_3.append(directors[2])

    for row in writer_col:
        writers = row.split(",")
        if len(writers) < 3:
            for i in range((3 - len(writers))):
                writers.append("")
        writers_1.append(writers[0])
        writers_2.append(writers[1])
        writers_3.append(writers[2])

    for row in genre_col:
        genres = str(row).split(",")
        if len(genres) < 3:
            for i in range((3 - len(genres))):
                genres.append("")
        genres_1.append(genres[0])
        genres_2.append(genres[1])
        genres_3.append(genres[2])

    df_after_split_stars = pd.DataFrame(
        {"name": df["name"], "year": df["year"], "idbm_year": df['idbm_year'], "length": df["length"],
         "director_1": directors_1,"director_2": directors_2,"director_3": directors_3,
         "writer_1": writers_1, "writer_2": writers_2, "writer_3": writers_3,
         "star_1": stars_1, "star_2": stars_2, "star_3": stars_3,
         "genre_1": genres_1,"genre_2": genres_2,"genre_3": genres_3, "rating": df["rating"]})
    df_after_split_stars.to_csv('C:\\Users\\haraz\\Desktop\\JupiterProjectDataScience\\movie_after_split_multiply_values.csv')
    return df_after_split_stars

In [422]:
def choose_only_main_column(df , columns_to_delete):
    for col in columns_to_delete:
        df = df.drop([col], axis=1).copy()
#     df.to_csv('C:\\Users\\haraz\\Desktop\\JupiterProjectDataScience\\movie_after_delete_extra_columns.csv')
    return df
    
    

In [423]:

df = divide_columns(df)
df = choose_only_main_column(df, ['director_2','director_3'])
df.head()



,name,year,idbm_year,length,director_1,writer_1,writer_2,writer_3,star_1,star_2,star_3,genre_1,genre_2,genre_3,rating
0,Adam at Six A.M.,1970,1970,1h 40m,Robert Scheerer,Stephen Karpf,Elinor Karpf,,Michael Douglas,Lee Purcell,Joe Don Baker,Drama,,,5.7
1,Adam's Woman,1970,1970,1h 56m,Philip Leacock,Richard Fielder,Lowell Barrington,,Beau Bridges,John Mills,Jane Merrow,Drama,,,6.3
2,The Adventurers,1970,1970,2h 51m,Lewis Gilbert,Michael Hastings,Lewis Gilbert,Harold Robbins,Charles Aznavour,Alan Badel,Candice Bergen,Action,Adventure,Drama,5.4
3,Airport,1970,1970,2h 17m,George Seaton,Arthur Hailey,George Seaton,,Burt Lancaster,Dean Martin,George Kennedy,Action,Drama,Thriller,6.6
4,Alex in Wonderland,1970,1970,1h 50m,Paul Mazursky,Paul Mazursky,Larry Tucker,,Donald Sutherland,Ellen Burstyn,Meg Mazursky,Comedy,Drama,,5.4


In [424]:
def add_gender_director_to_df(df, womens_list):
    gender_director = [] 
    for name_director in df.director_1:
        if name_director in womens_list:
            gender_director.append(0)
        else:
            gender_director.append(1) 

    df.insert(3, "gender_director", gender_director, True)
    



In [425]:
add_gender_director_to_df(df, womens_list)
# df = load_csv('C:\\Users\\haraz\\Desktop\\JupiterProjectDataScience\\movie_with_gender_director.csv')
df.head(5)

,name,year,idbm_year,gender_director,length,director_1,writer_1,writer_2,writer_3,star_1,star_2,star_3,genre_1,genre_2,genre_3,rating
0,Adam at Six A.M.,1970,1970,1,1h 40m,Robert Scheerer,Stephen Karpf,Elinor Karpf,,Michael Douglas,Lee Purcell,Joe Don Baker,Drama,,,5.7
1,Adam's Woman,1970,1970,1,1h 56m,Philip Leacock,Richard Fielder,Lowell Barrington,,Beau Bridges,John Mills,Jane Merrow,Drama,,,6.3
2,The Adventurers,1970,1970,1,2h 51m,Lewis Gilbert,Michael Hastings,Lewis Gilbert,Harold Robbins,Charles Aznavour,Alan Badel,Candice Bergen,Action,Adventure,Drama,5.4
3,Airport,1970,1970,1,2h 17m,George Seaton,Arthur Hailey,George Seaton,,Burt Lancaster,Dean Martin,George Kennedy,Action,Drama,Thriller,6.6
4,Alex in Wonderland,1970,1970,1,1h 50m,Paul Mazursky,Paul Mazursky,Larry Tucker,,Donald Sutherland,Ellen Burstyn,Meg Mazursky,Comedy,Drama,,5.4


In [426]:
df.to_csv('movie_wiki_done.csv')